In [53]:
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
from huggingface_hub import hf_hub_download

In [130]:
# маленький сабсэмпл, чтобы не убивать ноут
SUBSAMPLE = "up0.001_ip0.001"
LOCAL_DIR = Path("VK-LSVD")
LOCAL_DIR.mkdir(exist_ok=True)

# сколько недель train использовать (можно увеличить, если память позволяет)
N_WEEKS_TRAIN = 25

train_files = [f"subsamples/{SUBSAMPLE}/train/week_{i:02}.parquet" for i in range(N_WEEKS_TRAIN)]
val_files   = [f"subsamples/{SUBSAMPLE}/validation/week_25.parquet"]

meta_files = [
    "metadata/users_metadata.parquet",
    "metadata/items_metadata.parquet",
]

SUBMISSION_PATH = Path("submission.parquet")

In [131]:
all_files = train_files + val_files + meta_files

for file in all_files:
    local_path = LOCAL_DIR / file
    if local_path.exists():
        continue
    print("Скачиваю:", file)
    hf_hub_download(
        repo_id="deepvk/VK-LSVD",
        repo_type="dataset",
        filename=file,
        local_dir=str(LOCAL_DIR),
    )

print("Готово, всё нужное скачано.")

Готово, всё нужное скачано.


In [132]:
# train читаем стримингом
train_scans = [pl.scan_parquet(LOCAL_DIR / f) for f in train_files]
train = pl.concat(train_scans).collect(engine="streaming")

print("train shape:", train.shape)
print("train columns:", train.columns)

users_meta = pl.read_parquet(LOCAL_DIR / "metadata/users_metadata.parquet")
items_meta = pl.read_parquet(LOCAL_DIR / "metadata/items_metadata.parquet")

print("users_meta shape:", users_meta.shape)
print("items_meta shape:", items_meta.shape)
print("items_meta columns:", items_meta.columns)

# читаем шаблон сабмита
sub_template = pl.read_parquet(SUBMISSION_PATH)
print("submission schema:", sub_template.schema)
print(sub_template.head())

# приведём всё к колонке item_id
if "item_id" in sub_template.columns:
    submission_items = sub_template.select("item_id")
elif "clip_id" in sub_template.columns:
    submission_items = sub_template.rename({"clip_id": "item_id"}).select("item_id")
else:
    raise ValueError("В шаблоне submission нет ни 'item_id', ни 'clip_id'.")

item_ids = submission_items["item_id"].to_list()
print("Клипов в сабмите:", len(item_ids))

train shape: (47068641, 12)
train columns: ['user_id', 'item_id', 'place', 'platform', 'agent', 'timespent', 'like', 'dislike', 'share', 'bookmark', 'click_on_author', 'open_comments']
users_meta shape: (10000000, 5)
items_meta shape: (19627601, 4)
items_meta columns: ['item_id', 'author_id', 'duration', 'train_interactions_rank']
submission schema: Schema([('item_id', UInt32), ('user_id', Array(UInt32, shape=(100,)))])
shape: (5, 2)
┌─────────┬─────────────────────────────────┐
│ item_id ┆ user_id                         │
│ ---     ┆ ---                             │
│ u32     ┆ array[u32, 100]                 │
╞═════════╪═════════════════════════════════╡
│ 3118    ┆ [453232773, 86990582, … 116953… │
│ 26853   ┆ [207024839, 310318706, … 10241… │
│ 41654   ┆ [452381271, 348753378, … 18567… │
│ 55168   ┆ [51348190, 413822332, … 140959… │
│ 65689   ┆ [386411529, 442835260, … 70264… │
└─────────┴─────────────────────────────────┘
Клипов в сабмите: 47765


In [133]:
# какие сигналы считаем сильными
fb_cols = ["like", "share", "bookmark", "click_on_author", "open_comments"]

cols_needed = ["user_id", "item_id"] + [c for c in fb_cols if c in train.columns]

train_small = train.select(cols_needed)

# strong = есть хотя бы один из этих сигналов
val_expr = pl.any_horizontal(*[c for c in fb_cols if c in train_small.columns])
train_strong = train_small.filter(val_expr)

print("Всего строк в train:", train_small.shape[0])
print("Строк со strong-интеракциями:", train_strong.shape[0])

# берём item_id -> author_id из метаданных
items_author = items_meta.select(["item_id", "author_id"])

# добавляем author_id к каждому strong-событию
train_strong_auth = train_strong.join(
    items_author,
    on="item_id",
    how="inner",
)

print("train_strong_auth shape:", train_strong_auth.shape)
print(train_strong_auth.head())

Всего строк в train: 47068641
Строк со strong-интеракциями: 1284933
train_strong_auth shape: (1284933, 8)
shape: (5, 8)
┌───────────┬───────────┬───────┬───────┬──────────┬─────────────────┬───────────────┬───────────┐
│ user_id   ┆ item_id   ┆ like  ┆ share ┆ bookmark ┆ click_on_author ┆ open_comments ┆ author_id │
│ ---       ┆ ---       ┆ ---   ┆ ---   ┆ ---      ┆ ---             ┆ ---           ┆ ---       │
│ u32       ┆ u32       ┆ bool  ┆ bool  ┆ bool     ┆ bool            ┆ bool          ┆ u32       │
╞═══════════╪═══════════╪═══════╪═══════╪══════════╪═════════════════╪═══════════════╪═══════════╡
│ 361619201 ┆ 147183736 ┆ false ┆ true  ┆ false    ┆ false           ┆ false         ┆ 9344      │
│ 506241144 ┆ 147183736 ┆ false ┆ true  ┆ false    ┆ false           ┆ false         ┆ 9344      │
│ 310439257 ┆ 147183736 ┆ false ┆ true  ┆ false    ┆ false           ┆ false         ┆ 9344      │
│ 99631671  ┆ 147183736 ┆ false ┆ true  ┆ false    ┆ false           ┆ false         ┆ 9

In [134]:
# считаем число strong-интеракций на (author_id, user_id)
author_user_counts = (
    train_strong_auth
    .group_by(["author_id", "user_id"])
    .len()
    .rename({"len": "cnt"})
)

print("author_user_counts shape:", author_user_counts.shape)
print(author_user_counts.head())

# строим словарь author_id -> [user_id,...] по убыванию cnt
author_to_user_counts = {}

for row in author_user_counts.iter_rows(named=True):
    a = int(row["author_id"])
    u = int(row["user_id"])
    c = int(row["cnt"])
    author_to_user_counts.setdefault(a, []).append((u, c))

author_to_users = {}
for a, lst in author_to_user_counts.items():
    lst_sorted = sorted(lst, key=lambda x: -x[1])
    author_to_users[a] = [u for (u, c) in lst_sorted]

print("Число авторов:", len(author_to_users))
some_author = next(iter(author_to_users.keys()))
print("Пример автора:", some_author)
print("Топ-10 его юзеров:", author_to_users[some_author][:10])

# глобально популярные юзеры (по числу strong-интеракций)
user_counts = (
    train_strong
    .group_by("user_id")
    .len()
    .rename({"len": "cnt"})
    .sort("cnt", descending=True)
)

global_popular_users = user_counts["user_id"].to_list()
print("Всего юзеров со strong-интеракциями:", len(global_popular_users))
print("Топ-10 глобально популярных:", global_popular_users[:10])

author_user_counts shape: (887642, 3)
shape: (5, 3)
┌───────────┬───────────┬─────┐
│ author_id ┆ user_id   ┆ cnt │
│ ---       ┆ ---       ┆ --- │
│ u32       ┆ u32       ┆ u32 │
╞═══════════╪═══════════╪═════╡
│ 267221    ┆ 108747183 ┆ 1   │
│ 317643    ┆ 293207322 ┆ 1   │
│ 1185617   ┆ 486021682 ┆ 1   │
│ 351301    ┆ 225875139 ┆ 1   │
│ 213874    ┆ 68663581  ┆ 1   │
└───────────┴───────────┴─────┘
Число авторов: 4890
Пример автора: 267221
Топ-10 его юзеров: [285876492, 481768941, 281543257, 454647607, 90607025, 156100100, 452381271, 452440570, 247058089, 425659427]
Всего юзеров со strong-интеракциями: 9786
Топ-10 глобально популярных: [285876492, 202749663, 180557436, 408627999, 90607025, 246205628, 223500221, 433458222, 70679223, 401129130]


In [135]:
# оставляем только тех item_id, которые есть в submission
items_author_sub = items_author.filter(
    pl.col("item_id").is_in(item_ids)
)

item_to_author = {
    int(row["item_id"]): int(row["author_id"])
    for row in items_author_sub.iter_rows(named=True)
}

print("item_to_author size:", len(item_to_author))
some_item = next(iter(item_to_author.keys()))
print("Пример item_id -> author_id:", some_item, "->", item_to_author[some_item])

item_to_author size: 47765
Пример item_id -> author_id: 88620250 -> 558


In [136]:
# полный пул всех пользователей из метаданных
all_users = users_meta["user_id"].to_list()
print("Всего пользователей в users_metadata:", len(all_users))

K = 100

user_to_count = {}           # user_id -> сколько раз уже рекомендован (глобально)
item_to_pred_users = {}      # item_id -> список предсказанных user_id

# глобальный указатель по all_users для fallback
all_users_pos = 0
n_all = len(all_users)

def get_next_fallback_user():
    """Крутимся по all_users по кругу, пока не найдём юзера с лимитом < 100."""
    global all_users_pos
    for _ in range(n_all):
        u = all_users[all_users_pos]
        all_users_pos = (all_users_pos + 1) % n_all
        if user_to_count.get(u, 0) < 100:
            return u
    raise RuntimeError("Все пользователи выбили лимит 100 рекомендаций (что почти невозможно при реальных масштабах).")

for item_id in item_ids:
    item = int(item_id)

    preds = []
    used_local = set()

    # 1) авторские юзеры
    author = item_to_author.get(item)
    if author is not None and author in author_to_users:
        for u in author_to_users[author]:
            if user_to_count.get(u, 0) >= 100:
                continue
            if u in used_local:
                continue
            preds.append(u)
            used_local.add(u)
            user_to_count[u] = user_to_count.get(u, 0) + 1
            if len(preds) >= K:
                break

    # 2) глобально популярные юзеры
    if len(preds) < K:
        for u in global_popular_users:
            if user_to_count.get(u, 0) >= 100:
                continue
            if u in used_local:
                continue
            preds.append(u)
            used_local.add(u)
            user_to_count[u] = user_to_count.get(u, 0) + 1
            if len(preds) >= K:
                break

    # 3) fallback: крутимся по всем пользователям по кругу
    while len(preds) < K:
        u = get_next_fallback_user()
        if u in used_local:
            continue
        preds.append(u)
        used_local.add(u)
        user_to_count[u] = user_to_count.get(u, 0) + 1

    # на всякий случай проверяем
    assert len(preds) == K, f"Не удалось набрать 100 юзеров для item {item}"

    item_to_pred_users[item] = preds

len(item_to_pred_users), len(user_to_count)

Всего пользователей в users_metadata: 10000000


(47765, 3807686)

In [138]:
submissions_df = pl.DataFrame({
    "item_id": item_ids,
    "user_id": [item_to_pred_users[int(i)] for i in item_ids]
})

print(submissions_df.head())
print(submissions_df.shape)

shape: (5, 2)
┌─────────┬─────────────────────────────────┐
│ item_id ┆ user_id                         │
│ ---     ┆ ---                             │
│ i64     ┆ list[i64]                       │
╞═════════╪═════════════════════════════════╡
│ 3118    ┆ [285876492, 202749663, … 17497… │
│ 26853   ┆ [285876492, 481768941, … 99426… │
│ 41654   ┆ [285876492, 202749663, … 17497… │
│ 55168   ┆ [285876492, 202749663, … 17497… │
│ 65689   ┆ [285876492, 202749663, … 17497… │
└─────────┴─────────────────────────────────┘
(47765, 2)


In [139]:
def check(df: pl.DataFrame):
    ex = df.explode('user_id')
    ex_i = ex.group_by('item_id').len()
    ex_u = ex.group_by('user_id').len()
    # нет дублей (item_id, user_id)
    assert (
        ex.group_by('item_id', 'user_id')
        .len()
        .select('len')
        .max()
        .to_numpy()[0][0] == 1
    ), "doubles"
    # ровно 100 рекомендаций на item
    assert ex_i.select('len').min().to_numpy()[0][0] == 100, "le recs on item"
    assert ex_i.select('len').max().to_numpy()[0][0] == 100, "ge recs on item"
    # каждый пользователь не встречается более 100 раз
    assert ex_u.select('len').max().to_numpy()[0][0] == 100, "users capacity"

check(submissions_df)
print("✅ check() прошёл успешно, сабмит удовлетворяет всем ограничениям.")

✅ check() прошёл успешно, сабмит удовлетворяет всем ограничениям.


In [140]:
submissions_df.write_parquet("submission_01.parquet")

In [141]:
val = pl.read_parquet(LOCAL_DIR / f"subsamples/{SUBSAMPLE}/validation/week_25.parquet")

print(val.shape)
print(val.columns)

(519839, 12)
['user_id', 'item_id', 'place', 'platform', 'agent', 'timespent', 'like', 'dislike', 'share', 'bookmark', 'click_on_author', 'open_comments']


In [142]:
fb_cols = ["like", "share", "bookmark", "open_comments", "click_on_author"]

val_strong = val.filter(
    pl.any_horizontal(*fb_cols)
)

gt_df = (
    val_strong
    .group_by("item_id")
    .agg(pl.col("user_id").unique().alias("gt_users"))
)

print("Клипов с хотя бы одним релевантным юзером:", gt_df.height)

Клипов с хотя бы одним релевантным юзером: 3895


In [143]:
import numpy as np

def dcg_at_k(relevances, k=100):
    relevances = np.asarray(relevances)[:k]
    if relevances.size == 0:
        return 0.0
    discounts = 1.0 / np.log2(np.arange(2, relevances.size + 2))
    return float(np.sum(relevances * discounts))

def ndcg_item_at_k(pred_users, true_users, k=100):
    """
    pred_users: список user_id, наша выдача (до 100)
    true_users: множество user_id (все релевантные пользователи клипа)
    """
    true_users = set(true_users)
    R = len(true_users)

    if R == 0:
        # по формуле формально IDCG=0, но в практике обычно такие клипы не учитывают
        return None

    # DCG по нашей выдаче
    rel = [1 if u in true_users else 0 for u in pred_users[:k]]
    dcg = dcg_at_k(rel, k)

    # IDCG: будто все R релевантных стоят сверху
    ideal_rels = [1] * min(R, k)
    idcg = dcg_at_k(ideal_rels, k)

    if idcg == 0.0:
        return None

    return dcg / idcg

In [144]:
val_pred_users = {}

for row in gt_df.iter_rows(named=True):
    item = int(row["item_id"])
    preds = []
    used = set()

    # 1) author-based
    author = item_to_author.get(item)
    if author is not None and author in author_to_users:
        for u in author_to_users[author]:
            if u in used:
                continue
            preds.append(u)
            used.add(u)
            if len(preds) >= 100:
                break

    # 2) global popular
    if len(preds) < 100:
        for u in global_popular_users:
            if u in used:
                continue
            preds.append(u)
            used.add(u)
            if len(preds) >= 100:
                break

    # 3) fallback всеми юзерами
    if len(preds) < 100:
        for u in all_users:
            if u in used:
                continue
            preds.append(u)
            used.add(u)
            if len(preds) >= 100:
                break

    val_pred_users[item] = preds

In [145]:
scores = []

for row in gt_df.iter_rows(named=True):
    item = int(row["item_id"])
    true_users = row["gt_users"]
    preds = val_pred_users[item]

    score = ndcg_item_at_k(preds, true_users, k=100)
    if score is not None:
        scores.append(score)

print("Клипов, по которым посчитали NDCG:", len(scores))
print("item-NDCG@100 (правильная формула):", float(np.mean(scores)))

Клипов, по которым посчитали NDCG: 3895
item-NDCG@100 (правильная формула): 0.025539239589467796
